In [23]:
import pandas as pd
import numpy as np
import re
import os
import time
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import nltk
import faiss

In [22]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   --- ------------------------------------ 1.3/14.9 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/14.9 MB 5.9 MB/s eta 0:00:03
   ------- -------------------------------- 2.9/14.9 MB 4.4 MB/s eta 0:00:03
   --------- ------------------------------ 3.4/14.9 MB 4.0 MB/s eta 0:00:03
   ---------- ----------------------------- 3.9/14.9 MB 3.6 MB/s eta 0:00:04
   ----------- ---------------------------- 4.5/14.9 MB 3.3 MB/s eta 0:00:04
   ------------- -------------------------- 5.0/14.9 MB 3.3 MB/s eta 0:00:03
   --------------- ------------------------ 5.8/14.9 MB 3.3 MB/s eta 0:00:03
   ---------------- ----------------------- 6.3/14.9 MB 3.2 MB/s eta 0:00:03
   ------------------ --------------------- 6.8/14.9 MB 3.2 MB/s eta 0:00:03
   ------------------- -------------------- 7.1/14.9 MB 3.2 MB/s eta 0:00:03
   ----------

In [15]:
ls

 El volumen de la unidad C es OS
 El n�mero de serie del volumen es: 14C8-5E4E

 Directorio de c:\Users\karen\Documents\HumanidadesDigitales_git\C�digos

11/08/2025  00:08    <DIR>          .
09/08/2025  16:05    <DIR>          ..
09/08/2025  15:59    <DIR>          archivos_txt
09/08/2025  16:07        28.552.361 archivos_txt.zip
28/07/2025  00:18        30.437.819 columnas_procesadas.json
09/08/2025  15:58        21.306.801 Corpus_completo.xlsx
11/08/2025  05:41             5.983 InformationRetrieval.ipynb
28/07/2025  16:52           122.055 lda_visualization.html
28/07/2025  01:52        20.545.144 probs_bert.json
28/07/2025  01:52       197.568.767 topic_model.pkl
09/08/2025  16:07           211.212 topic_modeling.ipynb
28/07/2025  01:52           116.254 topics_bert.json
               9 archivos    298.866.396 bytes
               3 dirs  862.397.616.128 bytes libres


In [25]:
file_path = "Corpus_completo.xlsx"
corpus_completo = pd.read_excel(file_path)


In [26]:
# Formatear fechas

meses = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

def convertir_fecha(fecha_str):
    partes = fecha_str.split(' de ')
    dia = partes[0].zfill(2)  # Asegura 2 dígitos para días 1-9
    mes = meses[partes[1]]
    año = partes[2]
    return f"{dia}/{mes}/{año}"  # Formato DD/MM/AAAA

corpus_completo['Fecha_formateada'] = corpus_completo['Fecha'].apply(convertir_fecha)
corpus_completo['Fecha_formateada'] = pd.to_datetime(corpus_completo['Fecha_formateada'], format='%d/%m/%Y', errors='coerce')

In [32]:
# Generar embeddings de textos
modelo_embedding = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')


In [47]:
textos = corpus_completo['Texto'][:100].astype(str).tolist()  
embeddings = modelo_embedding.encode(textos, convert_to_tensor=False).astype('float32')


In [45]:
textos[:5]

['La Coalición Colombia \x96Partido Alianza Verde, Polo Democrático Alternativo y Compromiso Ciudadano- presentó por fin a Sergio Fajardo como su candidato único a la Presidencia.\n\nFajardo diagnostica a Colombia como un país fracturado, indignado por la corrupción, despistado por no tener una agenda de desarrollo clara luego del boom minero energético, y \x93desambientado\x94 en referencia al manejo de los recursos ambientales ( http://sergiofajardo.co/ ).\n\nPromete un liderazgo hacia la reconciliación, una lucha intensa contra la corrupción y mejor educación como mecanismo para el fomento de una sociedad innovadora y emprendedora que pueda usar eficientemente sus capacidades. Reconoce abiertamente que la desigualdad -a nivel de las personas y de las regiones- es un obstáculo grande para una Colombia incluyente.\n\nLa energía y el énfasis de sus presentaciones sobre su visión del país inspiran y al mismo tiempo lo exponen, en ciertas ocasiones, a críticas por no presentar los detall

In [48]:
# Crear índice FAISS
indice = faiss.IndexFlatL2(embeddings.shape[1])
indice.add(embeddings)

In [57]:
# Buscar texto similar a una consulta
consulta = "ciencia  y astronomía"
embedding_consulta = modelo_embedding.encode([consulta], convert_to_tensor=False).astype('float32')
distancias, indices = indice.search(embedding_consulta, k=5)

In [58]:
distancias

array([[8.151764, 8.541235, 8.653584, 9.070438, 9.205504]], dtype=float32)

In [55]:
indices[0]  # Índices de los textos más similares

array([49, 43, 84, 34, 36], dtype=int64)

In [59]:
#Resultados
print("Textos más similares:")
for idx in indices[0]:
    print(f"Índice: {idx}")
    print("Texto:" + textos[idx])


Textos más similares:
Índice: 34
Texto:Luego del espirituoso período del cambio de año, se avecina el desierto. El puente festivo de Reyes será la última ocasión para dorar las pieles. Siguen dos meses opacos durante los cuales la política azotará como monotema. Ni un fin de semana alargado más, ninguna golosina.

Como consolación para bobos, el calendario gregoriano previó que uno de esos dos meses fuera febrero, raquítico lapso de 28 días. Su brío no alcanza siquiera a completar la cifra reglamentaria. Sus cuatro semanas a duras penas coronan la aritmética del siete por cuatro.

La estación final de este aburrimiento bimestral serán las elecciones del 11 de marzo, que dos semanas más tarde harán naufragar el ánimo bajo el manto morado de la semana santa.

Así las cosas, a preparar los oídos para la mojiganga de los candidatos. Ninguno de ellos admitirá su rencor contra la paz, pues sus asesores de discursos ignoran cómo desconocer los tres mil muertos que hoy nadie llora porque gozan